In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

def rmsle(y_true, y_pred):
    """Root Mean Squared Logarithmic Error"""
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2))

In [2]:
# 전처리(Label Encoding)되 데이터 로드
X_train = pd.read_csv('TestData/train_data_oh_scaled.csv')
y_train = pd.read_csv('TestData/train_target.csv').iloc[:, 0]
X_val = pd.read_csv('TestData/val_data_label_encoded.csv')
y_val = pd.read_csv('TestData/val_target.csv').iloc[:, 0]


# 학습에 사용하지 않을 컬럼 삭제
X_train.drop(['id', 'date', 'date_type', 'day'], axis=1, inplace=True)
X_val.drop(['id', 'date', 'date_type', 'day'], axis=1, inplace=True)

In [3]:
# input, target, pca 주성분 개수를 입력 받아 dataframe의 주성분 분석을 통해, n개의 주성분으로 구성된 새로운 데이터 세트를 만들어 내는 함수

import pandas as pd
from sklearn.decomposition import PCA

def convert_pca_n_df (df_x, df_y, n):
    pca = PCA(n_components=df_x.shape[1])  # 모든 성분을 유지
    X_pca = pca.fit_transform(df_x)
    
    # 성분별 설명 분산 비율
    # explained_variance = pca.explained_variance_ratio_
    
    # PCA 성분들과 sales 사이의 상관관계 확인
    pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(df_x.shape[1])])
    pca_df['sales'] = df_y.values
    
    # 성분들과 sales 간의 상관계수 계산
    correlations = pca_df.corr()['sales'].drop('sales').abs()
    
    # sales에 가장 큰 영향을 주는 성분들을 선택
    important_components = correlations.sort_values(ascending=False)
    
    # 상위 몇 개의 성분을 기준으로 features 축소
    n_important = n  # 상위 5개의 중요한 성분을 선택하는 경우
    selected_components = important_components.index[:n_important]
    df_result = pca_df[selected_components]
    
    return df_result
    

In [4]:
# 추출할 주성분 개수를 n에 지정
n=10

# One-Hot Encoded & Standard Scaled 된 X_train, X_val 데이터를 주성분 분석하여 n 개의 주성분으로 구성된 새로운 데이터 생성 및 그 결과를 csv 파일로 저장
X_train_pca = convert_pca_n_df(X_train, y_train, n)
X_val_pca = convert_pca_n_df(X_val, y_val, n)

X_train_pca.to_csv(f'TestData/X_train_pca_{n}.csv', index=False)
X_val_pca.to_csv(f'TestData/X_train_pca_{n}.csv', index=False)

##############################################  아래 코드들은 테스트 코드로 삭제 예정  ###################################################################

In [25]:
# 주성분 분석
from sklearn.decomposition import PCA

# PCA를 n_components=0.5로 적용 (50% 이상의 분산을 설명하는 성분 선택)
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train)

pd.DataFrame(X_train_pca).head()


,0,1,2,3,4,5,6,7,8,9
0,1.035323,0.414791,1.104362,-0.143888,0.432948,0.444113,-0.246095,-0.520241,-0.664156,-0.036802
1,1.035375,0.414663,1.104375,-0.143876,0.432952,0.444109,-0.246095,-0.520240,-0.664153,-0.036840
2,1.035236,0.415001,1.104340,-0.143907,0.432941,0.444121,-0.246096,-0.520243,-0.664160,-0.036738
3,1.025678,0.438183,1.101950,-0.146052,0.432220,0.444953,-0.246208,-0.520412,-0.664587,-0.029714
4,1.035376,0.414662,1.104375,-0.143876,0.432952,0.444109,-0.246095,-0.520240,-0.664153,-0.036841


In [18]:
all_data = pd.concat([X_train, y_train], axis=1)
corr_matrix = all_data.corr()

# target 컬럼과 다른 컬럼들 간의 상관계수 추출
target_corr = corr_matrix['sales']

print(target_corr)

onpromotion    0.422753
dcoilwtico    -0.074811
year_2013     -0.067329
year_2014     -0.015817
year_2015      0.007610
                 ...   
cluster_14     0.090040
cluster_15    -0.045729
cluster_16    -0.014798
cluster_17     0.029315
sales          1.000000
Name: sales, Length: 270, dtype: float64


In [21]:
import pandas as pd
from sklearn.decomposition import PCA


# PCA 적용
pca = PCA(n_components=X_train.shape[1])  # 모든 성분을 유지
X_pca = pca.fit_transform(X_train)

# 성분별 설명 분산 비율
explained_variance = pca.explained_variance_ratio_

# PCA 성분들과 sales 사이의 상관관계 확인
pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_train.shape[1])])
pca_df['sales'] = y_train

# 성분들과 sales 간의 상관계수 계산
correlations = pca_df.corr()['sales'].drop('sales').abs()

# sales에 가장 큰 영향을 주는 성분들을 선택
important_components = correlations.sort_values(ascending=False)

print("Sales와 상관계수가 높은 PCA 성분들:")
print(important_components)

# 상위 몇 개의 성분을 기준으로 features 축소
n_important = 5  # 상위 5개의 중요한 성분을 선택하는 경우
selected_components = important_components.index[:n_important]
X_reduced = pca_df[selected_components]

print("축소된 Feature들:")
print(X_reduced.head())

Sales와 상관계수가 높은 PCA 성분들:
PC90     0.451958
PC2      0.348800
PC89     0.301548
PC1      0.253720
PC229    0.167299
           ...   
PC202    0.000031
PC192    0.000025
PC203    0.000023
PC188    0.000019
PC152    0.000006
Name: sales, Length: 269, dtype: float64
축소된 Feature들:
       PC90       PC2      PC89       PC1         PC229
0 -0.049330  0.414791 -0.065128  1.035323  1.720846e-15
1 -0.051014  0.414663 -0.066330  1.035375  1.831868e-15
2 -0.046553  0.415001 -0.063146  1.035236  1.720846e-15
3  0.259004  0.438183  0.154925  1.025678 -1.831868e-15
4 -0.051033  0.414662 -0.066343  1.035376  1.942890e-15
